In [1]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import glob

import math
from scipy import linalg
from numpy.linalg import inv
from sklearn import linear_model, datasets

In [45]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

In [46]:
objp.shape

(42, 3)

In [47]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        
        # termination criteria
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (7,6), corners,ret)
        cv2.imshow('img',img)
        cv2.waitKey(5000)
    
cv2.destroyAllWindows()

In [48]:
print(corners.shape)

(42, 1, 2)


In [6]:
imgp = np.ones((3,42))

imgp1 = np.array(imgpoints)
imgp1 = imgp1.reshape((2,42))
imgp[:2,:] = imgp1
print(imgp.shape)

objp = objp.T
print(objp.shape)
obj = np.ones((4,42))
obj[:3,:] = objp

(3, 42)
(3, 42)


In [7]:
obj

array([[0., 1., 2., 3., 4., 5., 6., 0., 1., 2., 3., 4., 5., 6., 0., 1.,
        2., 3., 4., 5., 6., 0., 1., 2., 3., 4., 5., 6., 0., 1., 2., 3.,
        4., 5., 6., 0., 1., 2., 3., 4., 5., 6.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
        2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 4., 4., 4., 4.,
        4., 4., 4., 5., 5., 5., 5., 5., 5., 5.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [44]:
imgp

array([[146.39855957, 240.75753784, 148.83552551, 209.49371338,
        151.51916504, 180.04098511, 154.09693909, 152.48091125,
        156.50086975, 126.54233551, 158.92370605, 102.02474213,
        161.30209351,  78.79632568, 177.50489807, 240.6578064 ,
        179.30621338, 208.87289429, 180.81489563, 179.32913208,
        182.54638672, 151.49290466, 184.25480652, 125.36465454,
        185.76144409, 100.64969635, 187.34809875,  77.39798737,
        209.64189148, 240.22723389, 210.3638916 , 208.30288696,
        210.88798523, 178.40151978, 211.65026855, 150.34837341,
        212.41748047, 124.12636566, 213.21379089,  99.37349701,
        213.91741943,  76.18415833],
       [242.29850769, 239.72090149, 241.74269104, 207.66035461,
        241.53677368, 177.55673218, 241.34033203, 149.31512451,
        241.14352417, 123.0146637 , 240.99368286,  98.10075378,
        240.77990723,  74.75045013, 275.28555298, 239.39506531,
        273.88638306, 206.89265442, 272.51751709, 176.77252197,
   

We now have the image and object points

Arranging the matrix in "THAT" form

In [10]:
A = []

In [11]:
for i in range(42):
    A.append(np.array([obj[0,i], obj[1][i], obj[2][i],1,0,0,0,0, (-1)*obj[0,i]*imgp[0,i], (-1)*obj[1][i]*imgp[0,i], (-1)*obj[2][i]*imgp[0,i],(-1)*imgp[0,i]]))
    A.append(np.array([0,0,0,0, obj[0,i], obj[1][i], obj[2][i],1, (-1)*obj[0,i]*imgp[1,i], (-1)*obj[1][i]*imgp[1,i], (-1)*obj[2][i]*imgp[1,i],(-1)*imgp[1,i]]))

In [12]:
A

[array([   0.        ,    0.        ,    0.        ,    1.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
          -0.        ,   -0.        ,   -0.        , -146.39855957]),
 array([   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    1.        ,
          -0.        ,   -0.        ,   -0.        , -242.29850769]),
 array([   1.        ,    0.        ,    0.        ,    1.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
        -240.75753784,   -0.        ,   -0.        , -240.75753784]),
 array([   0.        ,    0.        ,    0.        ,    0.        ,
           1.        ,    0.        ,    0.        ,    1.        ,
        -239.72090149,   -0.        ,   -0.        , -239.72090149]),
 array([   2.        ,    0.        ,    0.        ,    1.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
        -297.67105103,   -0.        ,   

In [13]:
len(A)

84

In [14]:
a = np.array(A)
# d.reshape((12,12))
a.shape

(84, 12)

In [15]:
ata = np.matmul(a.T,a)

In [16]:
u1, s1, vh1 = np.linalg.svd(ata, full_matrices=True)

print(u1.shape)
print(s1.shape)
print(vh1.shape)

(12, 12)
(12,)
(12, 12)


In [17]:
s1

array([6.38388575e+07, 1.11079758e+07, 5.95474849e+05, 8.24900952e+02,
       1.40784737e+02, 1.06617366e+02, 1.21780160e+01, 7.45177883e+00,
       4.61419255e-01, 2.69901747e-12, 4.42141557e-17, 2.59766512e-48])

In [18]:
vh1[:,8]

array([ 7.36272397e-01,  6.63321944e-01, -1.33769582e-01,  3.44934227e-05,
       -2.03496154e-04,  2.86038742e-03, -2.07351591e-03, -1.35368601e-05,
       -3.61457770e-04, -1.00979315e-15,  3.13196161e-19, -1.42537158e-34])

In [19]:
p1 = vh1[:,8]

In [20]:
p1.reshape((3,4))

array([[ 7.36272397e-01,  6.63321944e-01, -1.33769582e-01,
         3.44934227e-05],
       [-2.03496154e-04,  2.86038742e-03, -2.07351591e-03,
        -1.35368601e-05],
       [-3.61457770e-04, -1.00979315e-15,  3.13196161e-19,
        -1.42537158e-34]])

Now trying the same things after excluding the Z

In [21]:
C = []

In [22]:
for i in range(42):
    C.append(np.array([obj[0,i], obj[1][i],1,0,0,0, (-1)*obj[0,i]*imgp[0,i], (-1)*obj[1][i]*imgp[0,i],(-1)*imgp[0,i]]))
    C.append(np.array([0,0,0, obj[0,i], obj[2][i],1, (-1)*obj[0,i]*imgp[1,i], (-1)*obj[1][i]*imgp[1,i],(-1)*imgp[1,i]]))

In [23]:
len(C)

84

In [24]:
C = np.array(C)
C.shape

(84, 9)

In [25]:
ctc = np.matmul(C.T,C)
# ctcinv = inv(ctc)
# ctcinv.shape

In [26]:
print(ctc.shape)
# print(ctc)

(9, 9)


In [27]:
u, s, vh = np.linalg.svd(ctc, full_matrices=True)

In [28]:
print(u.shape)
print(s.shape)
print(vh.shape)

(9, 9)
(9,)
(9, 9)


In [29]:
s

array([6.38386866e+07, 1.11079410e+07, 5.95474741e+05, 7.18901267e+02,
       1.22487149e+02, 6.34988795e+01, 7.57293184e+00, 7.83158023e-01,
       7.06770810e-15])

In [30]:
vh[:,8]

array([ 2.09672640e-01, -3.56838750e-02, -9.77114111e-01,  1.14633591e-05,
        4.76688485e-04, -9.37282096e-04, -5.52605631e-05, -3.30919084e-03,
       -3.72436056e-18])

In [31]:
p = vh[:,8]

In [39]:
p = p.reshape((3,3))
pinv = inv(p)
pinv

array([[ 4.83988277e+00, -5.04556501e+03, -7.79002674e+02],
       [-8.08217659e-02,  8.42564776e+01, -2.89180020e+02],
       [ 1.80890140e-02, -1.08577240e+03, -1.56600423e+02]])

In [43]:
check = np.matmul(pinv,imgp)
check

array([[-1222603.32397519, -1209141.15791513, -1219787.12030071,
        -1047528.89754209, -1218735.16238083,  -895781.66115616,
        -1217731.52713827,  -753420.18112496, -1216726.88553621,
         -620845.03586792, -1215959.12522834,  -495258.94594973,
        -1214868.99524842,  -377555.89361738, -1388891.05453973,
        -1207497.61296549, -1381822.73357939, -1043658.42098757,
        -1374908.72950424,  -891828.32286758, -1369042.24013112,
         -749195.48359752, -1363112.57728787,  -615166.08324171,
        -1357961.02015529,  -489263.36804556, -1351805.9063139 ,
         -370959.75861249, -1557328.80189511, -1204169.83646744,
        -1545283.27083451, -1040505.0839897 , -1533475.20847814,
         -887167.34312996, -1521406.23170947,  -744178.16979895,
        -1510757.22011753,  -609609.52884128, -1500403.70626721,
         -483304.07408508, -1490702.1237003 ,  -365955.09848107],
       [   20114.20657582,    19889.40029741,    20067.17846277,
           17190.6183452